## Clustering and Segmentation
### By Agus Tri Wibowo 950220

### Import Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.offline as pyoff

import datetime

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, homogeneity_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs

from scipy.cluster.hierarchy import dendrogram, linkage


import warnings
import os
warnings.filterwarnings("ignore")

py.offline.init_notebook_mode(connected = True)

### Load Data, Exploration, and Cleansing

In [4]:
transaction_df = pd.read_csv('transactions.csv')

In [5]:
transaction_df.head()

,customer_id,trans_date,trans_amount
0,CS5295,11-Feb-18,35
1,CS4768,15-Mar-20,39
2,CS2122,26-Feb-18,52
3,CS1217,16-Nov-16,99
4,CS1850,20-Nov-18,78


In [7]:
transaction_df.dtypes

customer_id     object
trans_date      object
trans_amount     int64
dtype: object

In [8]:
transaction_df.shape

(125000, 3)

In [9]:
transaction_df['trans_date'] = transaction_df['trans_date'].replace(['29-Feb-17'],'28-Feb-17')

In [11]:
transaction_df['trans_date'] = pd.to_datetime(transaction_df['trans_date'])

In [12]:
transaction_df['trans_date'] = pd.to_datetime(transaction_df['trans_date'], format="%d%m%Y")

In [13]:
transaction_df.dtypes

customer_id             object
trans_date      datetime64[ns]
trans_amount             int64
dtype: object

In [14]:
transaction_df.duplicated().sum()

6

In [15]:
transaction_df=transaction_df.drop_duplicates()

In [16]:
transaction_df.duplicated().sum()

0

In [18]:
transaction_df.isnull().values.any()

False

In [19]:
transaction_df.dtypes

customer_id             object
trans_date      datetime64[ns]
trans_amount             int64
dtype: object

In [20]:
transaction_df

,customer_id,trans_date,trans_amount
0,CS5295,2018-02-11,35
1,CS4768,2020-03-15,39
2,CS2122,2018-02-26,52
3,CS1217,2016-11-16,99
4,CS1850,2018-11-20,78
...,...,...,...
124995,CS8433,2016-06-26,64
124996,CS7232,2019-08-19,38
124997,CS8731,2019-11-28,42
124998,CS8133,2018-12-14,13


### RFM Analysis

### Recency

In [22]:
df_rec = transaction_df.copy()

In [23]:
df_rec

,customer_id,trans_date,trans_amount
0,CS5295,2018-02-11,35
1,CS4768,2020-03-15,39
2,CS2122,2018-02-26,52
3,CS1217,2016-11-16,99
4,CS1850,2018-11-20,78
...,...,...,...
124995,CS8433,2016-06-26,64
124996,CS7232,2019-08-19,38
124997,CS8731,2019-11-28,42
124998,CS8133,2018-12-14,13


In [24]:
today = pd.to_datetime("now")

In [25]:
df_rec['days_from_last_purchase'] = (today-df_rec['trans_date']).astype('timedelta64[D]')

In [26]:
df_rec.head()

,customer_id,trans_date,trans_amount,days_from_last_purchase
0,CS5295,2018-02-11,35,1294.0
1,CS4768,2020-03-15,39,531.0
2,CS2122,2018-02-26,52,1279.0
3,CS1217,2016-11-16,99,1746.0
4,CS1850,2018-11-20,78,1012.0


In [33]:
recency = df_rec[['customer_id','days_from_last_purchase']].groupby('customer_id').min().reset_index()

In [34]:
recency.rename(columns={'days_from_last_purchase':'recency'}, inplace=True)

In [35]:
recency

,customer_id,recency
0,CS1112,592.0
1,CS1113,566.0
2,CS1114,563.0
3,CS1115,541.0
4,CS1116,734.0
...,...,...
6884,CS8996,628.0
6885,CS8997,792.0
6886,CS8998,615.0
6887,CS8999,788.0


In [36]:
recency.head()

,customer_id,recency
0,CS1112,592.0
1,CS1113,566.0
2,CS1114,563.0
3,CS1115,541.0
4,CS1116,734.0


### Frequency

In [37]:
df = df_rec.copy()

In [40]:
frequency = df[['customer_id','trans_date']].groupby(['customer_id']).count().reset_index()
frequency.rename(columns={'trans_date':'frequency'}, inplace=True)

frequency

,customer_id,frequency
0,CS1112,15
1,CS1113,20
2,CS1114,19
3,CS1115,22
4,CS1116,13
...,...,...
6884,CS8996,13
6885,CS8997,14
6886,CS8998,13
6887,CS8999,12


### Monetary

In [44]:
monetary = df[['customer_id','trans_amount']].groupby('customer_id').sum().reset_index()
monetary.rename(columns = {'trans_amount':'monetary'}, inplace = True)

monetary

,customer_id,monetary
0,CS1112,1012
1,CS1113,1490
2,CS1114,1432
3,CS1115,1659
4,CS1116,857
...,...,...
6884,CS8996,582
6885,CS8997,543
6886,CS8998,624
6887,CS8999,383


### RFM 

In [46]:
data = pd.merge(recency, frequency, on='customer_id')

In [47]:
data = pd.merge(data, monetary, on='customer_id')

In [48]:
data

,customer_id,recency,frequency,monetary
0,CS1112,592.0,15,1012
1,CS1113,566.0,20,1490
2,CS1114,563.0,19,1432
3,CS1115,541.0,22,1659
4,CS1116,734.0,13,857
...,...,...,...,...
6884,CS8996,628.0,13,582
6885,CS8997,792.0,14,543
6886,CS8998,615.0,13,624
6887,CS8999,788.0,12,383


In [49]:
data.head()

,customer_id,recency,frequency,monetary
0,CS1112,592.0,15,1012
1,CS1113,566.0,20,1490
2,CS1114,563.0,19,1432
3,CS1115,541.0,22,1659
4,CS1116,734.0,13,857


### Data Exploration

In [51]:
data.shape

(6889, 4)

In [53]:
data.describe()

,recency,frequency,monetary
count,6889.000000,6889.000000,6889.000000
mean,611.349252,18.143998,1179.223835
std,85.545443,5.193150,465.833636
min,530.000000,4.000000,149.000000
25%,553.000000,14.000000,781.000000
50%,584.000000,18.000000,1227.000000
75%,642.000000,22.000000,1520.000000
max,1388.000000,39.000000,2933.000000


## Segmenting and Clustering

In [59]:
data.head()

,customer_id,recency,frequency,monetary,no,rfm
0,CS1112,592.0,15,1012,1619.0,1619.0
1,CS1113,566.0,20,1490,2076.0,2076.0
2,CS1114,563.0,19,1432,2014.0,2014.0
3,CS1115,541.0,22,1659,2222.0,2222.0
4,CS1116,734.0,13,857,1604.0,1604.0


In [62]:
kmeans = KMeans(n_clusters=3)
recency['recency_cluster'] = kmeans.fit_predict(recency[['recency']])
recency.head()

,customer_id,recency,recency_cluster
0,CS1112,592.0,0
1,CS1113,566.0,0
2,CS1114,563.0,0
3,CS1115,541.0,0
4,CS1116,734.0,2


In [65]:
kmeans=KMeans(n_clusters=3)
frequency['frequency_cluster']=kmeans.fit_predict(frequency[['frequency']])
frequency.head()

,customer_id,frequency,frequency_cluster
0,CS1112,15,0
1,CS1113,20,0
2,CS1114,19,0
3,CS1115,22,1
4,CS1116,13,2


In [67]:
kmeans = KMeans(n_clusters=4)
monetary['monetary_cluster'] = kmeans.fit_predict(monetary[['monetary']])
monetary.head()

,customer_id,monetary,monetary_cluster
0,CS1112,1012,2
1,CS1113,1490,0
2,CS1114,1432,0
3,CS1115,1659,3
4,CS1116,857,2


In [68]:
data = pd.merge(recency, frequency, on='customer_id')
data = pd.merge(data, monetary, on='customer_id')
data

,customer_id,recency,recency_cluster,frequency,frequency_cluster,monetary,monetary_cluster
0,CS1112,592.0,0,15,0,1012,2
1,CS1113,566.0,0,20,0,1490,0
2,CS1114,563.0,0,19,0,1432,0
3,CS1115,541.0,0,22,1,1659,3
4,CS1116,734.0,2,13,2,857,2
...,...,...,...,...,...,...,...
6884,CS8996,628.0,2,13,2,582,1
6885,CS8997,792.0,1,14,2,543,1
6886,CS8998,615.0,0,13,2,624,1
6887,CS8999,788.0,1,12,2,383,1


In [69]:
data['rfm_score'] = data['recency_cluster'] + data['frequency_cluster'] + data['monetary_cluster']

In [70]:
data.groupby('rfm_score')['recency','frequency','monetary'].mean()

,recency,frequency,monetary
rfm_score,,,
0,565.957234,19.571429,1396.445860
1,577.059756,20.408537,1244.746341
2,605.556380,17.747774,1177.820475
3,611.096892,14.034735,735.278793
4,627.126471,20.461176,1407.665882
5,693.125000,11.878906,604.373047
6,666.405063,20.905063,1499.110759


#### Segmenting

In [71]:
data['segment'] = 'Rendah'
data.loc[data['rfm_score']>1,'segment'] = 'Menengah' 
data.loc[data['rfm_score']>4,'segment'] = 'Tinggi' 

In [72]:
data

,customer_id,recency,recency_cluster,frequency,frequency_cluster,monetary,monetary_cluster,rfm_score,segment
0,CS1112,592.0,0,15,0,1012,2,2,Menengah
1,CS1113,566.0,0,20,0,1490,0,0,Rendah
2,CS1114,563.0,0,19,0,1432,0,0,Rendah
3,CS1115,541.0,0,22,1,1659,3,4,Menengah
4,CS1116,734.0,2,13,2,857,2,6,Tinggi
...,...,...,...,...,...,...,...,...,...
6884,CS8996,628.0,2,13,2,582,1,5,Tinggi
6885,CS8997,792.0,1,14,2,543,1,4,Menengah
6886,CS8998,615.0,0,13,2,624,1,3,Menengah
6887,CS8999,788.0,1,12,2,383,1,4,Menengah


In [74]:
data.count()

customer_id          6889
recency              6889
recency_cluster      6889
frequency            6889
frequency_cluster    6889
monetary             6889
monetary_cluster     6889
rfm_score            6889
segment              6889
dtype: int64

In [75]:
data['segment'].value_counts()

Menengah    4142
Rendah      1919
Tinggi       828
Name: segment, dtype: int64

## Conclusions

After utilizing K-Means Clustering to create various groupings (clusters) for each, unsupervised machine learning was used to identify them.
Thus, there are 3 type of customers as shown and explained below: 
- Rendah : Customers that are less engaged than others, are not regular buyers/visitors, and earn a little amount of revenue. There are about 4142 customers (about three-fifths of all customers), hence this type of customer should be given a loyalty program to boost their recency, frequency and monetory.
- Menengah : A very safe type of customer. This segment has moderately engagement (purchasement), frequency and monetory. This segment could be shifted to segment 3 with several strategies, overall I belive this segment is the easiest to maintain and to be improved. 
- Tinggi : with a total of 808 customers or just above a-fifth, this segment has High monetary and frequency. We should increase the loyalty, especially for this segment since they are most likely giving better revenue than other segments. It could be better pricing policies, better program, or prioritizing their needs. 